In [93]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path
from sqlalchemy import create_engine, text

In [94]:
#Reading in the source file for the project
   
#file_path = r"C:/Desktop/Analysis Projects/LOC Subset.csv"
file_path = 'wdl_data_en_reduced.csv'
# Specifying encoding as'ISO-8859-1' to read file
loc_df = pd.read_csv(file_path, encoding='utf-8')

#Prints the Dask DataFrame structure in which it prints the column headings and datatypes 
loc_df['Place'].unique()

C:\Users\Cait\AppData\Local\Temp\ipykernel_25004\2086962587.py:6: DtypeWarning: Columns (0,20,33,34,36,37,41,42,43,49,52,54,55,62,63,66,69,70,71,72,74,76,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  loc_df = pd.read_csv(file_path, encoding='utf-8')


array(['North America > United States of America > Maryland > Antietam',
       'Latin America and the Caribbean > Bolivia > La Paz',
       'East Asia > Japan > Hokkaido|Europe > Russian Federation > Sakhalin Oblast > Kuril Islands',
       ...,
       'East Asia > China > Zhejiang Province|East Asia > China > Zhejiang Province > Hangzhou',
       nan, 'Place'], dtype=object)

In [95]:
# dataframe for all the languages
lang_df = pd.DataFrame(loc_df['Language'])
lang_df.head()

,Language
0,NaN
1,NaN
2,Japanese
3,Japanese
4,Spanish


In [96]:
lang_df = lang_df['Language'].str.split('|', expand=True)
lang_df.head()

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Japanese,None,None,None,None,None,None,None
3,Japanese,None,None,None,None,None,None,None
4,Spanish,None,None,None,None,None,None,None


In [97]:
lang_df = lang_df.stack()
lang_df

2      0    Japanese
3      0    Japanese
4      0     Spanish
5      0       Dutch
       1     Spanish
              ...   
23861  1      Pushto
23862  0     Persian
       1      Pushto
23863  0     Persian
       1      Pushto
Length: 18379, dtype: object

In [98]:
lang_df = pd.DataFrame(lang_df)
lang_df.head()

0
2 0  Japanese
3 0  Japanese
4 0   Spanish
5 0     Dutch
  1   Spanish

In [99]:
lang_df.columns

RangeIndex(start=0, stop=1, step=1)

In [100]:
lang_df = lang_df.iloc[:,0].rename('Lang')


In [101]:
lang_df = pd.DataFrame(lang_df)
lang_df.head()

Lang
2 0  Japanese
3 0  Japanese
4 0   Spanish
5 0     Dutch
  1   Spanish

In [102]:
#lang_df = lang_df.dropna()
lang_df = lang_df['Lang'].unique()
lang_df

array(['Japanese', 'Spanish', 'Dutch', 'Latin', 'English', 'Russian',
       'Portuguese', 'German', 'Arabic', 'French', 'Chinese', 'Tagalog',
       'Aleut', 'Classical Nahuatl', 'Ottoman Turkish (1500-1928)',
       'Marathi', 'Sanskrit', 'Persian', 'Naxi', 'Omaha-Ponca', 'Italian',
       'Chagatai', 'Armenian', 'Bulgarian', 'Modern Greek (1453-)',
       'Belarusian', 'Polish', 'Georgian', 'Swedish', 'Nyoro',
       'Guerrero Nahuatl', 'Eastern Huasteca Nahuatl', 'Hebrew',
       'Central Nahuatl', 'Ladino', 'Old French (842-ca. 1400)', 'Mossi',
       'Dazaga', 'Czech', 'Serbo-Croatian', 'Church Slavic', 'Tuvalu',
       'Hindi', 'Burmese', 'Turkish', 'Slovak', 'Danish', 'Norwegian',
       'Hungarian', 'Javanese', 'Batak', 'Afrikaans', 'Tibetan', 'Manchu',
       'Mongolian', 'Ancient Greek (to 1453)', 'Tamil',
       'Northwest Oaxaca Mixtec', 'Kurdish', 'Scots',
       'Judeo-Yemeni Arabic', 'Judeo-Arabic', 'Mikasuki', 'Welsh',
       'Middle Cornish', 'Syriac', 'Middle High Ge

In [103]:
unique_lang = pd.DataFrame()
unique_lang['Lang'] = lang_df
unique_lang

,Lang
0,Japanese
1,Spanish
2,Dutch
3,Latin
4,English
...,...
109,Geez
110,Pushto
111,Old High German (ca. 750-1050)
112,Sichuan Yi


In [104]:
unique_lang.to_csv('lang_id.csv', index=False)

In [105]:
unique_lang['lang_id'] = unique_lang.index

In [106]:
# Export final language dataframe to CSV file
unique_lang.to_csv('languages_id.csv', index=False)

In [107]:
#Reading in the source file for the project
   
#file_path = r"C:/Desktop/Analysis Projects/LOC Subset.csv"
file_path2 = 'languages.csv'
# Specifying encoding as'ISO-8859-1' to read file
lang_df_or = pd.read_csv(file_path2, encoding='utf-8')
lang_df_or

FileNotFoundError: [Errno 2] No such file or directory: 'languages.csv'

In [ ]:
lang_df_or.merge(unique_lang, how="inner")
lang_df_or


,Lang,item_id,lang_id
0,Japanese,2,0
1,Japanese,3,1
2,Spanish,4,2
3,Dutch,5,3
4,Latin,6,4
...,...,...,...
18374,Russian,22097,18374
18375,Serbian,10164,18375
18376,Serbian,22097,18376
18377,Turkish,10164,18377


In [ ]:
lang_df_or.drop(columns='Lang',inplace=True)

In [ ]:
lang_df_or.head()

,item_id,lang_id
0,2,0
1,3,1
2,4,2
3,5,3
4,6,4


In [ ]:
lang_item = lang_df_or

In [ ]:
lang_item.head()

,item_id,lang_id
0,2,0
1,3,1
2,4,2
3,5,3
4,6,4


In [ ]:
lang_item.to_csv('lang_item.csv', index=False)